Q3

-60,000 training images

-10,000 test images

-each image 28x28

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from sklearn.decomposition import PCA
import os,codecs,numpy
import matplotlib

In [ ]:
import scipy.linalg as la
import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# PROVIDE YOUR DIRECTORY WITH THE EXTRACTED FILES HERE
datapath = '/content/drive/MyDrive/Colab Notebooks/SML/SML_A3/fmnist/'

files = os.listdir(datapath)

def get_int(b):   # CONVERTS 4 BYTES TO A INT
    return int(codecs.encode(b, 'hex'), 16)

data_dict = {}
for file in files:
    if file.endswith('ubyte'):  # FOR ALL 'ubyte' FILES
        print('Reading ',file)
        with open (datapath+file,'rb') as f:
            data = f.read()
            type = get_int(data[:4])   # 0-3: THE MAGIC NUMBER TO WHETHER IMAGE OR LABEL
            length = get_int(data[4:8])  # 4-7: LENGTH OF THE ARRAY  (DIMENSION 0)
            if (type == 2051):
                category = 'images'
                num_rows = get_int(data[8:12])  # NUMBER OF ROWS  (DIMENSION 1)
                num_cols = get_int(data[12:16])  # NUMBER OF COLUMNS  (DIMENSION 2)
                parsed = numpy.frombuffer(data,dtype = numpy.uint8, offset = 16)  # READ THE PIXEL VALUES AS INTEGERS
                parsed = parsed.reshape(length,num_rows,num_cols)  # RESHAPE THE ARRAY AS [NO_OF_SAMPLES x HEIGHT x WIDTH]           
            elif(type == 2049):
                category = 'labels'
                parsed = numpy.frombuffer(data, dtype=numpy.uint8, offset=8) # READ THE LABEL VALUES AS INTEGERS
                parsed = parsed.reshape(length)  # RESHAPE THE ARRAY AS [NO_OF_SAMPLES]                           
            if (length==10000):
                set = 'test'
            elif (length==60000):
                set = 'train'
            data_dict[set+'_'+category] = parsed  # SAVE THE NUMPY ARRAY TO A CORRESPONDING KEY

Reading  train-images-idx3-ubyte
Reading  t10k-labels-idx1-ubyte
Reading  t10k-images-idx3-ubyte
Reading  train-labels-idx1-ubyte


In [ ]:
dC0 = []
dC1 = []
dC2 = []
dC3 = []
dC4 = []
dC5 = []
dC6 = []
dC7 = []
dC8 = []
dC9 = []

dyC0 = []
dyC1 = []
dyC2 = []
dyC3 = []
dyC4 = []
dyC5 = []
dyC6 = []
dyC7 = []
dyC8 = []
dyC9 = []

for i in range(60000):
  if(data_dict['train_labels'][i] == 0): dC0.append(data_dict['train_images'][i])
  if(data_dict['train_labels'][i] == 1): dC1.append(data_dict['train_images'][i])
  if(data_dict['train_labels'][i] == 2): dC2.append(data_dict['train_images'][i])
  if(data_dict['train_labels'][i] == 3): dC3.append(data_dict['train_images'][i])
  if(data_dict['train_labels'][i] == 4): dC4.append(data_dict['train_images'][i])
  if(data_dict['train_labels'][i] == 5): dC5.append(data_dict['train_images'][i])
  if(data_dict['train_labels'][i] == 6): dC6.append(data_dict['train_images'][i])
  if(data_dict['train_labels'][i] == 7): dC7.append(data_dict['train_images'][i])
  if(data_dict['train_labels'][i] == 8): dC8.append(data_dict['train_images'][i])
  if(data_dict['train_labels'][i] == 9): dC9.append(data_dict['train_images'][i])

dC0 = np.transpose(np.array(dC0).reshape(len(dC0), 784))
dC1 = np.transpose(np.array(dC1).reshape(len(dC1), 784))
dC2 = np.transpose(np.array(dC2).reshape(len(dC2), 784))
dC3 = np.transpose(np.array(dC3).reshape(len(dC3), 784))
dC4 = np.transpose(np.array(dC4).reshape(len(dC4), 784))
dC5 = np.transpose(np.array(dC5).reshape(len(dC5), 784))
dC6 = np.transpose(np.array(dC6).reshape(len(dC6), 784))
dC7 = np.transpose(np.array(dC7).reshape(len(dC7), 784))
dC8 = np.transpose(np.array(dC8).reshape(len(dC8), 784))
dC9 = np.transpose(np.array(dC9).reshape(len(dC9), 784))


for i in data_dict['train_labels']:
  if(i==0): dyC0.append(i)
  if(i==1): dyC1.append(i)
  if(i==2): dyC2.append(i)
  if(i==3): dyC3.append(i)
  if(i==4): dyC4.append(i)
  if(i==5): dyC5.append(i)
  if(i==6): dyC6.append(i)
  if(i==7): dyC7.append(i)
  if(i==8): dyC8.append(i)
  if(i==9): dyC9.append(i)

Y_train = np.array(dyC0 + dyC1 + dyC2 + dyC3 + dyC4 + dyC5 + dyC6 + dyC7 + dyC8 + dyC9)

In [ ]:
X_train = np.concatenate((dC0, dC1, dC2, dC3, dC4, dC5, dC6, dC7, dC8, dC9), axis=1)

X_test = np.transpose(data_dict['test_images'].reshape(len(data_dict['test_images']), 784))
Y_test = data_dict['test_labels']

In [ ]:
u0 = dC0.mean(axis=1)
u1 = dC1.mean(axis=1)
u2 = dC2.mean(axis=1)
u3 = dC3.mean(axis=1)
u4 = dC4.mean(axis=1)
u5 = dC5.mean(axis=1)
u6 = dC6.mean(axis=1)
u7 = dC7.mean(axis=1)
u8 = dC8.mean(axis=1)
u9 = dC9.mean(axis=1)


In [ ]:
S0 = np.matmul(np.transpose(np.transpose(dC0)-u0), np.transpose(np.transpose(np.transpose(dC0)-u0)))
S1 = np.matmul(np.transpose(np.transpose(dC1)-u1), np.transpose(np.transpose(np.transpose(dC1)-u1)))
S2 = np.matmul(np.transpose(np.transpose(dC2)-u2), np.transpose(np.transpose(np.transpose(dC2)-u2)))
S3 = np.matmul(np.transpose(np.transpose(dC3)-u3), np.transpose(np.transpose(np.transpose(dC3)-u3)))
S4 = np.matmul(np.transpose(np.transpose(dC4)-u4), np.transpose(np.transpose(np.transpose(dC4)-u4)))
S5 = np.matmul(np.transpose(np.transpose(dC5)-u5), np.transpose(np.transpose(np.transpose(dC5)-u5)))
S6 = np.matmul(np.transpose(np.transpose(dC6)-u6), np.transpose(np.transpose(np.transpose(dC6)-u6)))
S7 = np.matmul(np.transpose(np.transpose(dC7)-u7), np.transpose(np.transpose(np.transpose(dC7)-u7)))
S8 = np.matmul(np.transpose(np.transpose(dC8)-u8), np.transpose(np.transpose(np.transpose(dC8)-u8)))
S9 = np.matmul(np.transpose(np.transpose(dC9)-u9), np.transpose(np.transpose(np.transpose(dC9)-u9)))


In [ ]:
Sw = S0 + S1 + S2 + S3 + S4 +  S5 + S6 + S7 + S8 + S9

In [ ]:
ux = X_train.mean(axis=1)

In [ ]:
St = np.matmul(np.transpose(np.transpose(X_train)-ux), np.transpose(np.transpose(np.transpose(X_train)-ux)))


In [ ]:
Sb = St - Sw


In [ ]:
mat = np.matmul(np.linalg.inv(Sw), Sb)

In [ ]:
eigenValues, eigenV = la.eig(mat)

In [ ]:
eigenValues = eigenValues.real
eigenV = eigenV.real  # As ASK FROM THE TA DEFINATELY WHAT SHOULD HAPPEN  HERE

In [ ]:
eigenV

array([[ 6.55559977e-01,  1.67691982e-01, -3.99085663e-01, ...,
         1.37865869e-01, -9.59429809e-01,  9.38817747e-01],
       [-5.65665408e-01,  1.85621577e-02,  5.62781442e-01, ...,
         5.63005794e-01,  8.03712267e-02, -1.43356040e-01],
       [ 2.72489559e-01, -1.54852207e-01, -4.41668793e-01, ...,
         1.23907422e-01, -1.09798533e-01,  1.07590661e-01],
       ...,
       [-1.11215905e-05, -1.48502539e-02, -1.03023044e-02, ...,
         2.86328547e-03,  2.47206412e-03, -6.71295678e-03],
       [-4.30188826e-03,  2.65469778e-02, -4.89137042e-03, ...,
         2.51238046e-02, -5.19321495e-03,  4.11713789e-03],
       [-1.30269384e-02, -3.22714028e-02, -6.02883507e-03, ...,
        -3.83229448e-02,  5.19920128e-02,  5.04888266e-03]])

In [ ]:
idx = eigenValues.argsort()[::-1]
eigenValues = eigenValues[idx]
eigenV = eigenV[:,idx]


In [ ]:
W = np.transpose(np.transpose(eigenV)[:][0:9])

In [ ]:
Y_pC0 = np.matmul(np.transpose(W), dC0)
Y_pC1 = np.matmul(np.transpose(W), dC1)
Y_pC2 = np.matmul(np.transpose(W), dC2)
Y_pC3 = np.matmul(np.transpose(W), dC3)
Y_pC4 = np.matmul(np.transpose(W), dC4)
Y_pC5 = np.matmul(np.transpose(W), dC5)
Y_pC6 = np.matmul(np.transpose(W), dC6)
Y_pC7 = np.matmul(np.transpose(W), dC7)
Y_pC8 = np.matmul(np.transpose(W), dC8)
Y_pC9 = np.matmul(np.transpose(W), dC9)


In [ ]:
Y_pt = np.concatenate((Y_pC0, Y_pC1, Y_pC2, Y_pC3, Y_pC4, Y_pC5, Y_pC6, Y_pC7, Y_pC8, Y_pC9), axis = 1)

In [ ]:
Y_ptt = np.matmul(np.transpose(W), X_test)

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import accuracy_score

In [ ]:
lda = LDA()
lda.fit(np.transpose(Y_pt), Y_train)

LinearDiscriminantAnalysis()

In [ ]:
Y_pred = lda.predict(np.transpose(Y_ptt))
print("accuracy:", accuracy_score(Y_test, Y_pred))

accuracy: 0.8151


In [ ]:
print("Class1 accuracy:", accuracy_score(Y_test[0:1000], Y_pred[0:1000]))
print("Class2 accuracy:", accuracy_score(Y_test[1000:2000], Y_pred[1000:2000]))
print("Class3 accuracy:", accuracy_score(Y_test[2000:3000], Y_pred[2000:3000]))
print("Class4 accuracy:", accuracy_score(Y_test[3000:4000], Y_pred[3000:4000]))
print("Class5 accuracy:", accuracy_score(Y_test[4000:5000], Y_pred[4000:5000]))
print("Class6 accuracy:", accuracy_score(Y_test[5000:6000], Y_pred[5000:6000]))
print("Class7 accuracy:", accuracy_score(Y_test[6000:7000], Y_pred[6000:7000]))
print("Class8 accuracy:", accuracy_score(Y_test[7000:8000], Y_pred[7000:8000]))
print("Class9 accuracy:", accuracy_score(Y_test[8000:9000], Y_pred[8000:9000]))
print("Class10 accuracy:", accuracy_score(Y_test[9000:10000], Y_pred[9000:10000]))

Class1 accuracy: 0.816
Class2 accuracy: 0.839
Class3 accuracy: 0.811
Class4 accuracy: 0.792
Class5 accuracy: 0.826
Class6 accuracy: 0.8
Class7 accuracy: 0.81
Class8 accuracy: 0.823
Class9 accuracy: 0.811
Class10 accuracy: 0.823
